# Line chart (benchmark analysis)

In [ ]:



gapminder = pd.read_csv('uni.csv',
                        header=0,
                        usecols=["University Name", "Arts and humanities ranking THE2024", "Engineering & Technology ranking THE24", "Natural science rankings QS24", "Life science/ medical rankings QS24", "Economics & Business Rankings THE24"])

@capture("graph")
def barchart(data_frame, x, y, title=None, text=None):
    fig_bar = px.bar(data_frame=data_frame, x=x, y=y, title=title, text=text)
    
    fig_bar.update_layout(barmode='stack', xaxis={'categoryorder':'total ascending'})
    fig_bar.update_layout(
    title=go.layout.Title(
    text="<sup>Smaller number means higher national ranking</sup>",
    xref="paper",
    x=0
        ),)

    return fig_bar


def create_subject():
    """Function returns a page to perform analysis on university level."""

    page_subject = vm.Page(
        title="Uni rankings by subject",
        description="Discovering how different NZ universities are ranked through the years",
        layout=vm.Layout( grid=[[1, 1, 1, 1, 1]] + [[0, 0, 0, 0, 0]] * 4,
            row_min_height="100px",
            row_gap="24px",),
        components=[
            vm.Graph(
                id="barchart",
                figure=barchart(
                    gapminder,
                    x="University Name",
                    y="Arts and humanities ranking THE2024",
                    text="Arts and humanities ranking THE2024"
                ),
            ),
            vm.Card(
            text="""
               scroll down! 
               ### National rankings of universities based on subject field
               
                 - Subject fields to choose from: 
                    - 🏛️ Arts and humanities: Art, History, Philosophy, Literature, Classics, Media Studies, Film studies 
                    - ⚙️ Engineering & Technology: Electronic, Computer engineering, Biomedical engineering, Computer Science, IT, Food Technology
                    - 🔬 Natural science: Physics, Chemistry, Biology 
                    - 🩺 Life science/ medical: Pharmaceuticals, Biotechnology, Biomedical Science 
                    - 🏭 Economics & Business: Economics, Marketing, Finance
                    
                >
                >  Choose subject field in selector on your left. 
                >
             
  
            
            """,
        ),
        ],
        controls=[
            vm.Parameter(
                targets=["barchart.y"],
                selector=vm.Dropdown(
                    options=["Arts and humanities ranking THE2024", "Engineering & Technology ranking THE24", "Natural science rankings QS24", "Life science/ medical rankings QS24", "Economics & Business Rankings THE24"], multi=False, value="Arts and humanities ranking THE2024", title="Choose subject field"
                ),
            ),
             vm.Parameter(
                targets=["barchart.text"],
                selector=vm.Dropdown(
                    options=["Arts and humanities ranking THE2024", "Engineering & Technology ranking THE24", "Natural science rankings QS24", "Life science/ medical rankings QS24", "Economics & Business Rankings THE24"], multi=False, value="Arts and humanities ranking THE2024", title="Choose subject field to see specific rankings on the bars"
                ),
            ),
        ],
    )

    return page_subject


IS_JUPYTERLAB = 'true'







In [28]:
import pandas as pd
import vizro.models as vm
import vizro.plotly.express as px
from vizro import Vizro
from vizro.models.types import capture

Vizro._reset()
gapminder = pd.read_csv('uni.csv', header=0, usecols=[
    "University Name", 
    "Academic reputation (QS25)", 
    "Teaching quality (THE24)", 
    "Employment Outcomes (QS25)", 
    "Research quality (THE24)", 
    "International Research Network (QS25)", 
    "Employer Reputation (QS25)"
])

gapminder_1 = pd.DataFrame(dict(
    r=[82.2, 39.7, 92.9, 88.3, 87.4, 44.7],
    r_otago=[45, 35.6, 59.1, 75.8, 79.5, 24.6],
    r_massey=[30.1, 28.7, 49.2, 60.5, 85.1, 14],
    r_victoria=[42.8, 28.6, 71.6, 69.4, 70.7, 21.4],
    r_waikato=[19.1, 26.1, 46.4, 74.3, 55.1, 11.7],
    r_canterbury=[34.7, 28.6, 82.3, 62.4, 63, 27.3],
    r_lincoln=[8.2, 33.9, 40, 68.4, 20.2, 5],
    r_aut=[19.3, 23.8, 12.4, 84, 52.6, 11.2],
    theta=['Academic reputation (QS25)', 'Teaching quality (THE24)', 'Employment Outcomes (QS25)', 'Research quality (THE24)', 'International Research Network (QS25)', 'Employer Reputation (QS25)']
))

@capture("graph")
def radarchart(data_frame, r, theta, title=None, markers=None, hover_name=None, line_close=None): 
    fig_radar = px.line_polar(data_frame=data_frame, r=r, theta=theta, title=title, markers=markers, hover_name=hover_name, line_close=line_close)
    
    fig_radar.update_layout(
        legend=dict(
            font=dict(
                family="Tahoma, sans-serif",
                size=12,
            )
        ),
        annotations=[
            dict(
                xref='paper', 
                yref='paper',
                x=0.5, 
                y=-0.1,  # Position below the chart
                showarrow=False,
                text="Source: <a href='https://www.topuniversities.com/universities/university-auckland'>QS Rankings</a>, <a href='https://www.timeshighereducation.com/world-university-rankings'>THE Rankings</a>",
                font=dict(
                    family="Tahoma, sans-serif",
                    size=12,
                ),
            )
        ]
    ) 
    
    fig_radar.update_traces(fill='toself')
    
    return fig_radar

def create_metrics():
    """Function returns a page to show scores of different metrics on each university."""
    page_years = vm.Page(
        title="Universities' Scores in metrics",
        description="Discovering how different NZ universities are scored with different metrics",
        layout=vm.Layout(grid=[[0],]),
        components=[
            vm.Tabs(
                tabs=[
                    vm.Container(
                        title="University of Auckland",
                        layout=vm.Layout(grid=[[1, 0, 0]]),
                        components=[
                            vm.Graph(
                                id="uoa",
                                figure=radarchart(
                                    gapminder_1,
                                    r="r",
                                    theta="theta",
                                    title="University of Auckland",
                                    markers=True,
                                    hover_name="theta",
                                    line_close=True
                                ),
                            ),
                            vm.Card(
                                text="""
                                ![](assets/images/uoa.png#my-image2)
                                
                                The University of Auckland is New Zealand’s leading university, located in the country’s largest city. 
                                It offers a comprehensive range of programs and is known for its strong emphasis on research and innovation.
                                
                                ### Highlighting the Strengths

                                    - Academic Reputation (82.2): UOA is globally recognized for its academic excellence, meaning you'll be learning from some of the best minds in the world.

                                    - Employment Outcomes (92.9): With such a high score, UOA graduates are in hot demand by employers. This translates to strong job prospects and exciting career opportunities right after graduation.

                                    - Research Quality (88.3): Dive into groundbreaking research at UOA, where innovative projects and discoveries are a daily norm. You'll be part of a university that pushes the boundaries of knowledge.

                                    - International Research Network (87.4): Collaborate with top researchers worldwide. UOA's strong international connections mean you'll gain a global perspective and invaluable experience.

                                Choosing the University of Auckland means opting for a university that's highly respected, offers excellent job outcomes, and excels in research. 
                                It's a place where you'll thrive academically and professionally, all while staying close to the vibrant life of Auckland. 🌟📚🌍

                                """,
                            ),
                        ],
                    ),
                    vm.Container(
                        title="University of Otago",
                        layout=vm.Layout(grid=[[1, 0, 0]]),
                        components=[
                            vm.Graph(
                                id="otago",
                                figure=radarchart(
                                    gapminder_1,
                                    r="r_otago",
                                    theta="theta",
                                    title="University of Otago",
                                    markers=True,
                                    hover_name="theta",
                                    line_close=True
                                ),
                            ),
                            vm.Card(
                                text="""
                                ![](assets/images/otago.png)
                                
                                The University of Otago is renowned for its vibrant campus life and strong focus on research excellence. 
                                Located in Dunedin, it offers a unique blend of academic rigor and a supportive community.
                                ### Highlighting the Strengths

                                    - Academic Reputation (45): Otago's academic reputation reflects its solid standing in the academic community, providing students with a respected degree.

                                    - Research Quality (75.8): Otago excels in research, offering students opportunities to engage in innovative projects and contribute to significant discoveries.

                                    - International Research Network (79.5): Otago's strong international collaborations mean students can participate in global research initiatives and build valuable connections.

                                Choosing the University of Otago means being part of a university that values academic excellence and offers rich research opportunities, all within a vibrant and supportive community. 🌟📚🌏

                                """,
                            ),
                        ],
                    ),
                    vm.Container(
                        title="Massey University",
                        layout=vm.Layout(grid=[[1, 0, 0]]),
                        components=[
                            vm.Graph(
                                id="massey",
                                figure=radarchart(
                                    gapminder_1,
                                    r="r_massey",
                                    theta="theta",
                                    title="Massey University",
                                    markers=True,
                                    hover_name="theta",
                                    line_close=True
                                ),
                            ),
                            vm.Card(
                                text="""
                                ![](assets/images/massey.png)
                                
                                Massey University is known for its innovative teaching methods and extensive research programs. 
                                With campuses in Palmerston North, Albany, and Wellington, it offers diverse learning environments.
                                ### Highlighting the Strengths

                                    - **International Research Network (85.1)**: Massey has an extensive international research network, allowing students to collaborate on global projects and gain international experience.

                                Choosing Massey University means embracing innovation and benefiting from a global perspective, with numerous opportunities for international collaboration. 🌟🌐📚

                                """,
                            ),
                        ],
                    ),
                    vm.Container(
                        title="Victoria University of Wellington",
                        layout=vm.Layout(grid=[[1, 0, 0]]),
                        components=[
                            vm.Graph(
                                id="victoria",
                                figure=radarchart(
                                    gapminder_1,
                                    r="r_victoria",
                                    theta="theta",
                                    title="Victoria University of Wellington",
                                    markers=True,
                                    hover_name="theta",
                                    line_close=True
                                ),
                            ),
                            vm.Card(
                                text="""
                                ![](assets/images/victoria.png)
                                
                                Victoria University of Wellington is celebrated for its strong emphasis on research and academic excellence. 
                                Located in New Zealand's capital, it offers unique opportunities for engagement with government and industry.
                                ### Highlighting the Strengths

                                    - **Employment Outcomes (71.6)**: Graduates from Victoria have strong employment prospects, with many finding successful careers shortly after graduation.

                                Choosing Victoria University of Wellington means gaining access to excellent academic programs and strong career opportunities in the heart of New Zealand's capital. 🌟🏙️📚

                                """,
                            ),
                        ],
                    ),
                    vm.Container(
                        title="University of Waikato",
                        layout=vm.Layout(grid=[[1, 0, 0]]),
                        components=[
                            vm.Graph(
                                id="waikato",
                                figure=radarchart(
                                    gapminder_1,
                                    r="r_waikato",
                                    theta="theta",
                                    title="University of Waikato",
                                    markers=True,
                                    hover_name="theta",
                                    line_close=True
                                ),
                            ),
                            vm.Card(
                                text="""
                                ![](assets/images/waikato.png)
                                
                                The University of Waikato is recognized for its focus on research and strong community engagement. 
                                Located in Hamilton, it offers a supportive and innovative learning environment.
                                ### Highlighting the Strengths

                                    - **Research Quality (74.3)**: Waikato's commitment to research excellence provides students with opportunities to participate in cutting-edge projects.

                                Choosing the University of Waikato means engaging in high-quality research and being part of a community that values innovation and support. 🌟🔬📚

                                """,
                            ),
                        ],
                    ),
                    vm.Container(
                        title="University of Canterbury",
                        layout=vm.Layout(grid=[[1, 0, 0]]),
                        components=[
                            vm.Graph(
                                id="canterbury",
                                figure=radarchart(
                                    gapminder_1,
                                    r="r_canterbury",
                                    theta="theta",
                                    title="University of Canterbury",
                                    markers=True,
                                    hover_name="theta",
                                    line_close=True
                                ),
                            ),
                            vm.Card(
                                text="""
                                ![](assets/images/canterbury.png)
                                
                                The University of Canterbury is known for its strong engineering and science programs. 
                                Located in Christchurch, it offers a robust academic environment and numerous research opportunities.
                                ### Highlighting the Strengths

                                    - **Employment Outcomes (82.3)**: Canterbury graduates are highly sought after by employers, ensuring strong career prospects.

                                Choosing the University of Canterbury means receiving a respected education with excellent job opportunities in a supportive academic community. 🌟🔧📚

                                """,
                            ),
                        ],
                    ),
                    vm.Container(
                        title="Lincoln University",
                        layout=vm.Layout(grid=[[1, 0, 0]]),
                        components=[
                            vm.Graph(
                                id="lincoln",
                                figure=radarchart(
                                    gapminder_1,
                                    r="r_lincoln",
                                    theta="theta",
                                    title="Lincoln University",
                                    markers=True,
                                    hover_name="theta",
                                    line_close=True
                                ),
                            ),
                            vm.Card(
                                text="""
                                ![](assets/images/lincoln.png)
                                
                                Lincoln University is a specialist land-based university known for its strong focus on agriculture and environmental science. 
                                Located near Christchurch, it offers unique programs tailored to these fields.
                                ### Highlighting the Strengths

                                    - **Research Quality (68.4)**: Lincoln excels in research, particularly in areas related to agriculture and environmental science.

                                Choosing Lincoln University means gaining expertise in specialized fields with strong research support, all while being close to nature. 🌟🌱📚

                                """,
                            ),
                        ],
                    ),
                    vm.Container(
                        title="AUT",
                        layout=vm.Layout(grid=[[1, 0, 0]]),
                        components=[
                            vm.Graph(
                                id="aut",
                                figure=radarchart(
                                    gapminder_1,
                                    r="r_aut",
                                    theta="theta",
                                    title="AUT",
                                    markers=True,
                                    hover_name="theta",
                                    line_close=True
                                ),
                            ),
                            vm.Card(
                                text="""
                                ![](assets/images/aut.png)
                                
                                Auckland University of Technology (AUT) is known for its innovative teaching and strong industry connections. 
                                Located in Auckland, it provides a dynamic learning environment with a focus on practical skills.
                                ### Highlighting the Strengths

                                    - **Research Quality (84)**: AUT's commitment to research excellence provides students with opportunities to engage in high-impact projects.

                                Choosing AUT means embracing innovation and benefiting from strong industry ties, ensuring a practical and forward-thinking education. 🌟💡📚

                                """,
                            ),
                        ],
                    ),
                ],
            )
        ]
    )
    return page_years

dashboard = vm.Dashboard(
    pages=[
        create_metrics()
    ],
    navigation=vm.Navigation(
        nav_selector=vm.NavBar(
            items=[
                vm.NavLink(
                    label="Rankings of subject field",
                    pages=["Universities' Scores in metrics"],
                    icon="Auto Stories",
                ),
            ]
        ),
    ),
)

if not IS_JUPYTERLAB:
    app = Vizro().build(dashboard)
    server = app.dash.server
    
    if __name__ == "__main__":  
        app.run(port=8080)
else:
    Vizro(assets_folder="assets").build(dashboard).run(port=8082)
